In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import multirtd.params as params
from multirtd.utils import plot_trajectory
from multirtd.planners.dubins_planner import DubinsPlanner
from multirtd.dynamics.dubins_model import Turtlebot
from multirtd.dynamics.diff_drive_dynamics import DiffDriveDynamics
from multirtd.sensors.position_sensor import PositionSensor

%load_ext autoreload
%autoreload 2

In [ ]:
dynamics = DiffDriveDynamics()
sensor = PositionSensor(n=2, sigma=0.1)

x0 = np.array([0, 0, 0])
u = np.array([0.2, 0.1])
N = 100
x_nom = dynamics.nominal_traj(x0, u, N)
u_nom = np.tile(u, (N, 1))

# Plot nominal trajectory
fig, ax = plt.subplots(figsize=(6,6))
plot_trajectory(ax, x_nom)
ax.axis('equal')
ax.grid(True)
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
from multirtd.reachability.dubins_reachability import dlqr_calculate

# Test EKF implementation
x = np.array([0.0, 0.0, 0.0])
x_est = np.array([0.0, 0.0, 0.0])
P = 0.1 * np.eye(3)

motion_sigma = dynamics.sigma  # Uncertainty in left/right wheel velocities
range_sigma = sensor.sigma
Q_motion = motion_sigma**2 * np.eye(2)
R_sense = range_sigma**2 * np.eye(2)

Q_lqr = np.eye(3)
R_lqr = np.eye(2)

x_traj = np.zeros((N, 3))
x_traj_est = np.zeros((N, 3))

for i in range(1, N):
    # Linearize about nominal trajectory
    A, B = dynamics.linearize(x_nom[i], u_nom[i])
    C = dynamics.noise_matrix(x_nom[i])
    H = sensor.linearize(x_nom[i])

    # True dynamics
    K = dlqr_calculate(A, B, Q_lqr, R_lqr)
    u = u_nom[i] + K @ (x_nom[i] - x_est)
    # vl = u[0] - u[1] * d / 2
    # vr = u[0] + u[1] * d / 2
    # x = diff_drive_step(x, vl, vr, motion_sigma, d, dt)
    x = dynamics.step(x, u)

    # Measurement
    z = sensor.get_measurement(x)
    
    # Predict step
    x_est = dynamics.step(x_est, u, sigma=0.0)
    P = A @ P @ A.T + C @ Q_motion @ C.T
    # Update step
    # H = get_beacon_jacobian(x_est, beacon_positions)
    K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R_sense)
    x_est = x_est + K @ (z - sensor.get_measurement(x_est, sigma=0.0))
    P = (np.eye(P.shape[0]) - K @ H) @ P

    x_traj[i] = x
    x_traj_est[i] = x_est

In [ ]:
# Plot nominal trajectory
fig, ax = plt.subplots(figsize=(6,6))
plot_trajectory(ax, x_nom, color='g')
plot_trajectory(ax, x_traj, color='k')
ax.axis('equal')
ax.grid(True)
ax.set_xlabel('x')
ax.set_ylabel('y')